# i.MX 8ULP AHAB with U-BOOT

This notebook describes how to build a bootable AHAB image with ELE firmware and U-BOOT bootloader, load it using the NXPUUU app and how to use NXPELE app. We are going to reproduce the flash_singleboot image created by the imx-mkimage tool.


## 1. Prerequisites

- SPSDK is needed with examples extension. `pip install spsdk[examples]` (Please refer to the installation documentation.)
- This demo was tested with i.MX93 EVK board

### 1.1 Images preparation

- to create resulting binary containing AHAB containers, we need to prepare the binaries
- in this section we reproduce the process which is done by the `imx-mkimage` tool as flash singleboot binary
- Obtain all the necessary binaries (upower, u-boot, bl31, ELE firmware) and put them into inputs directory

### 1.2 U-Boot

[Read the U-Boot documentation to understand the U-Boot build process](https://docs.u-boot.org/en/latest/board/nxp/imx8ulp_evk.html)
In order to use the nxpele app, U-Boot must be built with AHAB support. CONFIG_AHAB_BOOT=y
If you want to use the nxpele over fastboot, also multiplexing of console output to fastboot must be enabled by setting CONFIG_CONSOLE_MUX=y.

### 1.3 Requirements

Download these files from the `https://www.nxp.com/lgfiles/NMG/MAD/YOCTO/<package name>` and put them into the inputs directory.

- uPower
- u-boot binary (u-boot SPL and u-boot) built with AHAB support.
- bl31.bin binary (ARM Trusted Firmware)
- ELE firmware binary AHAB 
- i.MX8ULP EVK board


In [3]:
from spsdk.utils.jupyter_utils import YamlDiffWidget

# This env variable sets colored logger output to STDOUT
# Execute this cell to enable execution of the ! line magic
%env JUPYTER_SPSDK=1
%alias execute echo %l && %l
%alias_magic ! execute

env: JUPYTER_SPSDK=1
Created `%!` as an alias for `%execute`.


## 2. AHAB Image

The U-Boot image (flash.bin created by imx-mkimage) consists of two container sets - first container set is load to OCRAM using the SDPS protocol supported by ROM. The first container set contains U-Boot SPL binary, ELE firmware and uPower firmware. The second container set contains the full U-Boot and ATF image and is loaded to DDR by U-Boot SPL.

### 2.1 AHAB Template

We can generate the template using the nxpimage ahab get-template command. The command will generate a template. The template is a YAML file that contains the AHAB header and the AHAB container. The AHAB header contains the information about the image, such as the version, the number of containers, and the signature. The AHAB container contains the information about the image, such as the image type, the image version, the image size, and the image signature.

The following command generates the template:

```bash
nxpimage ahab get-template -f mimx8ulp -o ahab_template.yaml
```

### 2.2 Exporting of the AHAB image

The AHAB image can be exported using the nxpimage ahab export command. The command will create the AHAB image from the template. The following command creates the AHAB image:

```bash
nxpimage ahab export -c ahab_template.yaml 
```

### 2.3 U-Boot SPL image

U-Boot SPL image contains ELE firmware (Optional), uPower, and U-Boot SPL binary. It is the first container set that is loaded to OCRAM by ROM using SDPS protocol. Let's prepare the configuration from the template. 




In [5]:
YamlDiffWidget("inputs/mimx8ulp_ahab_uboot_spl.diffc").html

nxpimage ahab get-template -f mimx8ulp -o workspace/ahab_template.yaml --force 
Creating workspace/ahab_template.yaml template file.


### 2.3 U-Boot ATF Image

The secondary container set contains the U-Boot and ATF images. 

In [18]:
YamlDiffWidget("inputs/mimx8ulp_ahab_uboot_atf.diffc").html

nxpimage ahab get-template -f mimx8ulp -o workspace/ahab_template.yaml --force
Creating workspace/ahab_template.yaml template file.


## 3. AHAB Image Container set
Now we have the AHAB image with U-Boot SPL DDR and AHAB Image with U-Boot ATF container. We can use the nxpimage binary-image merge command to merge the AHAB images into one binary image. The following command merges the AHAB images:

```bash
nxpimage -v bootable-image merge -c u-boot-flash_template.yaml -o flash.bin
```

We have to prepare the configuration. In configuration file we just specify the target memory (in our case we are downloading the image using the nxpuuu so serial downloader is enough). 

In [16]:
YamlDiffWidget("inputs/mimx8ulp_ahab_bimg.diffc").html

nxpimage bootable-image get-templates -f mimx8ulp -o workspace --force
Creating workspace/bootimg_mimx8ulp_serial_downloader.yaml template file.
Creating workspace/bootimg_mimx8ulp_flexspi_nor.yaml template file.
Creating workspace/bootimg_mimx8ulp_emmc.yaml template file.
Creating workspace/bootimg_mimx8ulp_sd.yaml template file.
Creating workspace/bootimg_mimx8ulp_flexspi_nand.yaml template file.
Creating workspace/bootimg_mimx8ulp_recovery_spi.yaml template file.


In [8]:
VERBOSITY = "-v"
U_BOOT_FLASH_BOOT_CFG = "inputs/u-boot-bootable.yaml"
U_BOOT_FLASH_BOOT = "outputs/flash.bin"
%! nxpimage $VERBOSITY bootable-image merge --config $U_BOOT_FLASH_BOOT_CFG --output $U_BOOT_FLASH_BOOT

nxpimage -v bootable-image merge --config inputs/u-boot-bootable.yaml --output outputs/flash.bin 
INFO:spsdk.apps.nxpimage:Created Bootable Image:
Name:      Bootable Image for mimx8ulp
Starts:    0x0
Ends:      0x1ae3ff
Size:      Size: 1.7 MiB; 1,762,304 B
Alignment: 1 B
Execution Start Address: Not defined
Pattern:zeros
Memory type: MemoryType.SERIAL_DOWNLOADER
Revision: latest

INFO:spsdk.apps.nxpimage:Created Bootable Image memory map:

+==0x0000_0000= Bootable Image for mimx8ulp =====================+
|                   Size: 1.7 MiB; 1,762,304 B                   |
|   Memory type: MemoryType.SERIAL_DOWNLOADER Revision: latest   |
|                         Pattern: zeros                         |
|+==0x0000_0000= primary_image_container_set ===================+|
||                  Size: 193.5 kiB; 198,144 B                  ||
||                  AHAB Image for mimx8ulp_a2                  ||
||                        Pattern: zeros                        ||
||+==0x0000_0000= 

## 4. Image download
First we put the `iMX8ULP` into serial downloader mode SW5 (`01xx_xxxx`), and use `uuu` to upload the `flash.bin` containing U-Boot.

In [7]:
%! nxpuuu $VERBOSITY write -f mimx9352 -b emmc $U_BOOT_FLASH_BOOT

nxpuuu -v write -f mimx9352 -b emmc outputs/flash.bin 
SDPS: boot -f outputs/flash.bin


Done
